本实验使用Qwen2.5-1.5B模型 测试
仅用于学习不用于生成最终模型 - 因此数据量也较低

### 1. 数据处理 - Tokenizer
#### 1.1 加载数据 - Dataset

In [1]:
import pandas as pd
from datasets import Dataset
train = pd.read_csv("data/train.csv")[:500] # 仅前500条数据用于测试
train_ds = Dataset.from_pandas(train)
train_ds[:5]

{'text': ['现头昏口苦',
  '目的观察复方丁香开胃贴外敷神阙穴治疗慢性心功能不全伴功能性消化不良的临床疗效',
  '舒肝和胃消痞汤；功能性消化不良',
  '患者３ａ前咯血，被诊断为肺结核，住院４０余天时出现腹痛，经治疗好转，但时有发作，坚持服抗痨药３ａ后，因腹痛基本缓解，肺结核治愈而停药',
  '治疗组采用复方蜥蜴散不同微粒组合剂（密点麻蜥、炙黄芪、焦乌梅、炒白芍、三七、半枝莲等）治疗'],
 'label': ["{'口苦': '临床表现'}",
  "{'复方丁香开胃贴': '中医治疗', '心功能不全伴功能性消化不良': '西医诊断'}",
  "{'功能性消化不良': '西医诊断'}",
  "{'咯血': '临床表现', '肺结核': '西医诊断'}",
  "{'复方蜥蜴散': '方剂', '密点麻蜥': '中药', '炙黄芪': '中药', '焦乌梅': '中药', '炒白芍': '中药', '三七': '中药', '半枝莲': '中药'}"]}

#### 1.2 Tokenization
+ 加载tokenizer 
+ 定义process function：prompt方程
+ 处理dataset为 [input_id, attention_mask,labels]

In [2]:
from modelscope import AutoTokenizer
model_dir ="/Users/luyi/PythonProjects/Atomy/03LLM微调/命名实体微调实战/qwen/Qwen2-1___5B-Instruct" # 定义本地路径
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True) # 加载tokenizer

In [10]:
# 定义process function:
def process_func(example):
    MAX_LENGTH = 250
    
    instruction = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 以 json 格式输出, 如 {'口苦': '临床表现','肺结核': '西医诊断'} 注意: 1. 输出的每一行都必须是正确的json字符串. 2.找不到任何实体时, 输出"没有找到任何实体"."""
    instructions_messages= [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{example['text']}"}
    ]
    response_messages=[
            {"role": "assistant", "content":f"{example['label']}"},
        ]
    # Chat-template + tokenizer 处理
    instructions_chatTamplate = tokenizer.apply_chat_template(instructions_messages, tokenize=True, add_generation_prompt=False,return_dict=True)
    total_chatTamplate = tokenizer.apply_chat_template(instructions_messages + response_messages, tokenize=True, padding=True,truncation=True,max_length=MAX_LENGTH,add_generation_prompt=False,return_dict=True)

    # 凑labels的内容
    instruction_len = len(instructions_chatTamplate['input_ids'])
    labels = [-100] * instruction_len + total_chatTamplate['input_ids'][instruction_len:]
    input_ids = total_chatTamplate['input_ids']
    attention_mask = total_chatTamplate['attention_mask']
    
    # 限制最大长度做截断处理
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  

In [11]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names,num_proc=4) 
# 删除column_names 保证简洁性
# 多线程处理数据 加速数据处理速度

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

### 2. 训练模型
#### 2.1 加载模型

In [12]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_dir)
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法 

#### 2.2 定义Lora参数

In [13]:
# 导包
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Lora微调参数
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # 实现方式为生成式，因此选CAUSAL_LM 不选 TOKEN_CLASSIFICATION
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, 
    r=8, 
    lora_alpha=32,  
    lora_dropout=0.1,  
)
model = get_peft_model(model, config)

In [14]:
model.print_trainable_parameters() # 检查模型可训练参数大小

trainable params: 9,232,384 || all params: 1,552,946,688 || trainable%: 0.5945


### 2.3 定义Trainer参数 并开始训练

In [15]:
# 训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2-NER-Doctor", # 模型输出地址
    per_device_train_batch_size=4, # 每张显卡上 的 batch_size 一个GPU 就是4
    gradient_accumulation_steps=8, # 累计多少步更新一次参数，通常会比per_device_train_batch_size大
    logging_steps=10,
    num_train_epochs=2, # 训练的轮次
    save_steps=50,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
    fp16=False, # fp16需要N卡
    bf16=True, # mps 仅支持高精度训练 
)

In [17]:
#设置 Trainer 开始训练
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding="max_length",max_length=250)
)

trainer.train()

  0%|          | 0/30 [00:00<?, ?it/s]

{'loss': 0.3227, 'grad_norm': 0.9531846642494202, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.64}
{'loss': 0.1874, 'grad_norm': 1.2382848262786865, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.28}
{'loss': 0.1484, 'grad_norm': 0.7349944710731506, 'learning_rate': 0.0, 'epoch': 1.92}


/opt/anaconda3/envs/tunning/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /Users/luyi/PythonProjects/Atomy/03LLM微调/命名实体微调实战/qwen/Qwen2-1___5B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 2447.2422, 'train_samples_per_second': 0.409, 'train_steps_per_second': 0.012, 'train_loss': 0.21947674751281737, 'epoch': 1.92}


TrainOutput(global_step=30, training_loss=0.21947674751281737, metrics={'train_runtime': 2447.2422, 'train_samples_per_second': 0.409, 'train_steps_per_second': 0.012, 'total_flos': 1900185108480000.0, 'train_loss': 0.21947674751281737, 'epoch': 1.92})

In [ ]:
# 保存LoRA权重（仅几MB~几十MB，无需保存完整大模型）
trainer.model.save_pretrained("./lora_weights_qwen2.5")

/opt/anaconda3/envs/tunning/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /Users/luyi/PythonProjects/Atomy/03LLM微调/命名实体微调实战/qwen/Qwen2-1___5B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
